In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from PIL import Image
import os
import cv2
import zipfile
import random
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
path = "C:\\Users\\nicov\\Documents\\Capstone\\train"
for dir in os.listdir(path):
  img_path = os.path.join(path, dir)
  print(f"There are {len(os.listdir(img_path))} images of {dir}.")
print("")

path = "C:\\Users\\nicov\\Documents\\Capstone\\val"
for dir in os.listdir(path):
  img_path = os.path.join(path, dir)
  print(f"There are {len(os.listdir(img_path))} images of {dir}.")
print("")

There are 300 images of alpine sea holly.
There are 300 images of anthurium.
There are 300 images of artichoke.
There are 300 images of azalea.
There are 300 images of balloon flower.
There are 300 images of barberton daisy.
There are 300 images of bee balm.
There are 300 images of bird of paradise.
There are 300 images of bishop of llandaff.
There are 300 images of black orchid.
There are 300 images of black-eyed susan.
There are 300 images of blackberry lily.
There are 300 images of blanket flower.
There are 300 images of bolero deep blue.
There are 300 images of bougainvillea.
There are 300 images of bromelia.
There are 300 images of bunga bangkai.
There are 300 images of buttercup.
There are 300 images of californian poppy.
There are 300 images of camellia.
There are 300 images of canna lily.
There are 300 images of canterbury bells.
There are 300 images of cape flower.
There are 300 images of carnation.
There are 300 images of cautleya spicata.
There are 300 images of clematis.
Th

In [5]:
train_path = "C:\\Users\\nicov\\Documents\\Capstone\\train"
val_path = "C:\\Users\\nicov\\Documents\\Capstone\\val"
image_size = (224,224)
batch_size = 32

def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range = 20, 
                                     zoom_range = 0.2, 
                                     horizontal_flip = True,
                                     fill_mode = 'nearest'
                                     )

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=100,
                                                      class_mode='categorical',
                                                      target_size=(224, 224))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=100,
                                                                class_mode='categorical',
                                                                target_size=(224, 224))
  return train_generator, validation_generator

In [6]:
train_generator, validation_generator = train_val_generators(train_path, val_path)

Found 31200 images belonging to 104 classes.
Found 12157 images belonging to 104 classes.


In [7]:
#Create base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [8]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(100, 7, 7, 1280)


In [9]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [2]:
# model 1
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# dropout_layer = tf.keras.layers.Dropout(0.3)
# dense_layer1 = tf.keras.layers.Dense(1024, activation='relu')
# dense_layer2 = tf.keras.layers.Dense(104, activation='softmax')

# # Build the model
# inputs = tf.keras.Input(shape=(224, 224, 3))
# x = base_model(inputs, training=False)
# x = global_average_layer(x)
# x = dropout_layer(x)
# x = dense_layer1(x)
# x = dropout_layer(x)  # You can add dropout before the final dense layer for regularization
# outputs = dense_layer2(x)

# model = tf.keras.Model(inputs, outputs)
# model.summary()

# model.compile(optimizer="adam",
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

# history = model.fit(train_generator,
#                     epochs=5,
#                     validation_data=validation_generator)

In [17]:
# model 2
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# dropout_layer = tf.keras.layers.Dropout(0.3)
# dense_layer1 = tf.keras.layers.Dense(512, activation='relu')
# dense_layer2 = tf.keras.layers.Dense(128, activation='relu')
# dense_layer3 = tf.keras.layers.Dense(104, activation='softmax')

# # Build the model
# inputs = tf.keras.Input(shape=(224, 224, 3))
# x = base_model(inputs, training=False)
# x = global_average_layer(x)
# x = dropout_layer(x)
# x = dense_layer1(x)
# x = dense_layer2(x)
# x = dropout_layer(x)  # You can add dropout before the final dense layer for regularization
# outputs = dense_layer3(x)

# # model = tf.keras.Model(inputs, outputs)
# # model.summary()

# model.compile(optimizer="adam",
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

# history = model.fit(train_generator,
#                     epochs=10,
#                     validation_data=validation_generator)

In [12]:
# model 3
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dropout_layer = tf.keras.layers.Dropout(0.25)
dense_layer1 = tf.keras.layers.Dense(512, activation='relu')
dense_layer2 = tf.keras.layers.Dense(104, activation='softmax')

# Build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
x = dropout_layer(x)
x = dense_layer1(x)
x = dropout_layer(x)  # You can add dropout before the final dense layer for regularization
outputs = dense_layer2(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)           2257984   ['input_4[0][0]']             
 tional)                                                                                          
                                                                                                  
 global_average_pooling2d_2  (None, 1280)                 0         ['mobilenetv2_1.00_224[2][0]']
  (GlobalAveragePooling2D)                                                                        
                                                                                            

In [13]:
model.save("modelCapstone.h5")

c:\Users\nicov\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
